#### Find the average age of each age group in the fakefriends dataset

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
people = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("fakefriends-header.csv")

In [5]:
people.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



In [10]:
people.select("age","friends").show()

+---+-------+
|age|friends|
+---+-------+
| 33|    385|
| 26|      2|
| 55|    221|
| 40|    465|
| 68|     21|
| 59|    318|
| 37|    220|
| 54|    307|
| 38|    380|
| 27|    181|
| 53|    191|
| 57|    372|
| 54|    253|
| 56|    444|
| 43|     49|
| 36|     49|
| 22|    323|
| 35|     13|
| 45|    455|
| 60|    246|
+---+-------+
only showing top 20 rows



#### get avg number of friends for each age directly on the dataframe

In [12]:
people.groupBy("age").avg("friends").show()

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 31|            267.25|
| 65|             298.2|
| 53|222.85714285714286|
| 34|             245.5|
| 28|             209.1|
| 26|242.05882352941177|
| 27|           228.125|
| 44| 282.1666666666667|
| 22|206.42857142857142|
| 47|233.22222222222223|
| 52| 340.6363636363636|
| 40| 250.8235294117647|
| 20|             165.0|
| 57| 258.8333333333333|
| 54| 278.0769230769231|
| 48|             281.4|
| 19|213.27272727272728|
| 64| 281.3333333333333|
| 41|268.55555555555554|
| 43|230.57142857142858|
+---+------------------+
only showing top 20 rows



#### do the same thing using a temporary view and SQL query

In [15]:
people.createOrReplaceTempView("peopletab")
avgfrnds = spark.sql("SELECT age,mean(friends) FROM peopletab group by age")

In [18]:
#avgfrnds.take(5)
for age in avgfrnds.collect():
  print(age)

Row(age=31, mean(friends)=267.25)
Row(age=65, mean(friends)=298.2)
Row(age=53, mean(friends)=222.85714285714286)
Row(age=34, mean(friends)=245.5)
Row(age=28, mean(friends)=209.1)
Row(age=26, mean(friends)=242.05882352941177)
Row(age=27, mean(friends)=228.125)
Row(age=44, mean(friends)=282.1666666666667)
Row(age=22, mean(friends)=206.42857142857142)
Row(age=47, mean(friends)=233.22222222222223)
Row(age=52, mean(friends)=340.6363636363636)
Row(age=40, mean(friends)=250.8235294117647)
Row(age=20, mean(friends)=165.0)
Row(age=57, mean(friends)=258.8333333333333)
Row(age=54, mean(friends)=278.0769230769231)
Row(age=48, mean(friends)=281.4)
Row(age=19, mean(friends)=213.27272727272728)
Row(age=64, mean(friends)=281.3333333333333)
Row(age=41, mean(friends)=268.55555555555554)
Row(age=43, mean(friends)=230.57142857142858)
Row(age=37, mean(friends)=249.33333333333334)
Row(age=61, mean(friends)=256.22222222222223)
Row(age=35, mean(friends)=211.625)
Row(age=55, mean(friends)=295.53846153846155)
R

#### sorting ther results by age

In [20]:
people.groupBy("age").avg("friends").sort("age").show()

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



format by removing too many floating point values. here we use agg function to first aggregate results of group by and then apply multiple other functions on the resulting aggregation

In [21]:
from pyspark.sql import functions as func
people.groupBy("age").agg(func.round(func.avg("friends"), 2)).sort("age").show()

+---+----------------------+
|age|round(avg(friends), 2)|
+---+----------------------+
| 18|                343.38|
| 19|                213.27|
| 20|                 165.0|
| 21|                350.88|
| 22|                206.43|
| 23|                 246.3|
| 24|                 233.8|
| 25|                197.45|
| 26|                242.06|
| 27|                228.13|
| 28|                 209.1|
| 29|                215.92|
| 30|                235.82|
| 31|                267.25|
| 32|                207.91|
| 33|                325.33|
| 34|                 245.5|
| 35|                211.63|
| 36|                 246.6|
| 37|                249.33|
+---+----------------------+
only showing top 20 rows



giving an alias name to avg number of friends column as shown below. Note that all 3 functions - avg, round, alias are used inside main brackets of agg()

In [24]:
people.groupBy("age").agg(func.round(func.avg("friends"), 2).alias("avg_friends")).sort("age").show()

+---+-----------+
|age|avg_friends|
+---+-----------+
| 18|     343.38|
| 19|     213.27|
| 20|      165.0|
| 21|     350.88|
| 22|     206.43|
| 23|      246.3|
| 24|      233.8|
| 25|     197.45|
| 26|     242.06|
| 27|     228.13|
| 28|      209.1|
| 29|     215.92|
| 30|     235.82|
| 31|     267.25|
| 32|     207.91|
| 33|     325.33|
| 34|      245.5|
| 35|     211.63|
| 36|      246.6|
| 37|     249.33|
+---+-----------+
only showing top 20 rows



In [26]:
spark.stop()